# Parallelize Python

In [1]:
import jellyfish

In [3]:
s1 = "jellyfish"
s2 = "smellyfish"
jellyfish.jaro_winkler_similarity(s1, s2)

/var/folders/rg/q_yx8y8n14db7cnn4382r6kh0000gn/T/ipykernel_74037/3261149550.py:3: DeprecationWarning: getargs: The 'u' format is deprecated. Use 'U' instead.
  jellyfish.jaro_winkler_similarity(s1, s2)


0.8962962962962964

In [5]:
import os
import sys
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [11]:
from pyspark.sql import SparkSession
from pyspark.sql import Column
from pyspark.sql.types import StringType, IntegerType, BooleanType, FloatType
from pyspark.sql.functions import udf, col

In [7]:
spark = SparkSession.builder.master("local").appName("demo").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

24/10/15 11:05:33 WARN Utils: Your hostname, Matthews-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.8.188 instead (on interface en0)


In [12]:
data = [
    ("jellyfish", "smellyfish"),
    ("li", "lee"),
    ("luisa", "bruna"),
    ("hi", "hi"),
    (None, None)
]
df = spark.createDataFrame(data, ["word1", "word2"])

In [13]:
df.show()

+---------+----------+
|    word1|     word2|
+---------+----------+
|jellyfish|smellyfish|
|       li|       lee|
|    luisa|     bruna|
|       hi|        hi|
|     NULL|      NULL|
+---------+----------+



In [14]:
@udf(returnType=FloatType())
def jaro_winkler_similarity(s1, s2):
     return None if s1 == None or s2 == None else jellyfish.jaro_winkler_similarity(s1, s2)

In [15]:
df.withColumn("jw_similarity", jaro_winkler_similarity(df.word1, df.word2)).show()

+---------+----------+-------------+
|    word1|     word2|jw_similarity|
+---------+----------+-------------+
|jellyfish|smellyfish|    0.8962963|
|       li|       lee|    0.6111111|
|    luisa|     bruna|          0.6|
|       hi|        hi|          1.0|
|     NULL|      NULL|         NULL|
+---------+----------+-------------+

